# Multiple resonators on a chip, with possibility of having holes in the resonator core

Ruirements: gdspy library.
The design is generated in the following order:


   *   **Ground and contact pads**. THe idea is that you should only specify the coordinates of the contact pads; whether they should be rotated should be decided by the code itself (if the contact pad is further that 900 $\mu m$ away, it's rotated). This is done by the functions 
   
   *Generate_Pads (sample_size_along_x_axis, sample_size_along_y_axis, [d], [d1], [d2], [layer], [contact pad coordinates])*. 
   
   
   *   **TL**. Connects two contact pads from left to right. Generally, in this code everything that involves coplanar waveguides should be drawn from left to right.
   
   *Generate_TL (contact_pad_1, contact_pad2, d, d1, d2)*
   
    
   *   **Resonators** The position is specified by the left lower corner in (x,y) format. 
   
       *Generate_Resonators (initial_point_array, [circular_claw_radius], [gap], [resonator_frequency], [mode (up or down)], [layer])*.
   
   *   **DC line**. At this point the we add resonators and dc bias lines to the sample cell.
   
   *Generate_Loops* ([coordinates of the second end; the first end automatically gets connected to a contact pad], [modes], [d], [d1], [d2], [d for the contact pad], [d1 for the contact pad], [d2 for the contact pad], [array of arc coordinates], [array of arc angles], layer)
   
   
   *   **Ground grid**. Can be generated only after everything else is in place
   
   *Generate_Grid* (sample_size_along_x, sample_size_along_y, layer1, layer2)
   
 IF you want to change anything, unfortunately you'll have to do a restart.

In [1]:
!pip install gdspy

In [2]:
from design_res3 import *
from resonators3_1 import*
import gdspy
import random as rd

In [3]:
class Sample:
    
    #Содержит функции, генерирующие все площадки, TL, все джозефсоны, все петельки
    
    def __init__(self, number):
        self.name = 'sample' + str(number)
        self.cell = gdspy.Cell(self.name)
    
    def Generate_Pads(self, a, b, number, d_arr, d1_arr, d2_arr, layer, coords = []):
        self.Pads = Pads(a, b, number, d_arr, d1_arr, d2_arr, layer, coords)
        self.Pads_cell = gdspy.Cell('Pads')
        self.Pads_cell.add(self.Pads.Generate_Ground())
        self.cell.add(gdspy.CellReference(self.Pads_cell, (0, 0)))
        
    def Generate_TL(self, start_pad, finish_pad, d, d1, d2, layer):
        TL = Transmission_Line(self.Pads, start_pad, finish_pad, d, d1, d2, layer)
        self.tl = TL
        self.tl_cell1 = gdspy.Cell('TL1')
        self.tl_cell2 = gdspy.Cell('TL2')
        self.tl_cell3 = gdspy.Cell('TL3')
        self.tl_cell1.add(TL.GenerateTL(d, coords = [], rots = [])) 
        self.tl_cell2.add(TL.GenerateTL(d1, coords = [], rots = [])) 
        self.tl_cell3.add(TL.GenerateTL(d2, coords = [], rots = [])) 
        #self.cell.add([gdspy.CellReference(self.tl_cell, (0, 0))])
      
    def Generate_Resonators(self, start_points, freqs, modes, ws, ss, TL_inner, TL_vacuum, TL_ground, DEs):
        self.RESONATORS1 = gdspy.Cell('RESONATORS1')
        self.RESONATORS2 = gdspy.Cell('RESONATORS2')
        self.RESONATORS3 = gdspy.Cell('RESONATORS3')
        self.RESONATORS_RESTRICTED = gdspy.Cell('RESONATORS_RESTRICTED')
        #self.resonators = []
        #self.res_cells = []
        self.res_references = []
        for i, (start, freq, mode, w, s, TLi, TLv, TLg, DE) in enumerate(zip(start_points, freqs, modes, ws, ss, TL_inner, TL_vacuum, TL_ground, DEs)):
            #self.resonators.append(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE))
            #self.res_cells.append(gdspy.Cell('res' + str(i)))
            self.RESONATORS1.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate1(mode))
            self.RESONATORS2.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate2(mode))
            self.RESONATORS3.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate3(mode))
            self.RESONATORS_RESTRICTED.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate3(mode))
            #self.res_cells[i].add(self.resonators[i].Generate3(mode))
            
        COMBINATION = gdspy.Cell('COMBINATION')
        COMBINATION.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS3, (0,0)), gdspy.CellReference(self.tl_cell3, (0,0)), 'or'))
        CUT = gdspy.Cell('CUT')
        CUT.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS2, (0,0)), gdspy.CellReference(self.tl_cell2, (0,0)), 'or'))
        CUTOUT = gdspy.Cell('CUTOUT')
        CUTOUT.add(gdspy.boolean(gdspy.CellReference(COMBINATION, (0,0)), gdspy.CellReference(CUT, (0,0)), 'not'))
        FILL = gdspy.Cell('FILL')
        FILL.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS1, (0,0)), gdspy.CellReference(self.tl_cell1, (0,0)), 'or'))
        
        CUTOUTFILL = gdspy.Cell('CUTOUTFILL')
        CUTOUTFILL.add(gdspy.boolean(gdspy.CellReference(CUTOUT, (0,0)), gdspy.CellReference(FILL, (0,0)), 'or'))
        self.cell.add(gdspy.CellReference(CUTOUTFILL, (0, 0)))

        
                        
    def Generate_Grid(self, a, b, l1, l2):
        self.grid_cell = gdspy.Cell('Grid')
        res1 = gdspy.Rectangle((400, 400), (405, b - 400), layer = l1) 
        x = 447
        while x < a - 400:
            r1 = gdspy.Rectangle((x, 400), (x + 5, b - 400), layer = l1) 
            x += 47
            res1 = gdspy.boolean(res1, r1, 'or')
            #self.grid_cell.add(r1)
        y = 400
        res2 = gdspy.Rectangle((400, 400), (400, 405), layer = l2) 
        while y < b - 400:
            r1 = gdspy.Rectangle((400, y), (a - 400, y + 5), layer = l2) 
            y += 47
            res2 = gdspy.boolean(res2, r1, 'or')
            #self.grid_cell.add(r1)
        res2 = gdspy.boolean(res2, res1, 'not')
        for area in self.Pads.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for area in self.tl.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        area = gdspy.CellReference(self.RESONATORS_RESTRICTED, (0,0))
        res1 = gdspy.boolean(res1, area, 'not', layer = l1)
        res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        
        #res = gdspy.boolean(res, self.Pads_cell.get_polygonsets(), 'not')
        self.grid_h = res1
        self.grid_v = res2
        self.grid_cell.add(res1)
        self.grid_cell.add(res2)
        self.cell.add(gdspy.CellReference(self.grid_cell, (0, 0)))
    #self.Resonators
    #self.JJ
    #self.Loops
    #self.TL

In [4]:
TL_inner = 12
TL_vacuum = 2
TL_ground = 22
TL_total = TL_inner + 2*(TL_vacuum + TL_ground)
number = 90
w = 4
s = 21
DE = 11
positions = [] 
for j in range(3):
    for i in range(15):
        if j == 0:
            positions.append(coordinates(800 + 50 + 570*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(800 + 50 + 570*i, 3900 + TL_total/2 - j*1400))
        elif j == 1:
            positions.append(coordinates(660 + 580*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(660 + 580*i, 3900 + TL_total/2 - j*1400))
        elif j == 2:
            positions.append(coordinates(620 + 570*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(620 + 570*i, 3900 + TL_total/2 - j*1400)) 
w_arr = np.full(number, w) 
s_arr = np.full(number, s) 
f_arr = np.linspace(8e9, 9e9, number)
rd.shuffle(f_arr)
print(f_arr)
DE_arr = np.full(number, DE)
TL_inner_arr = np.repeat(TL_inner, number)
TL_vacuum_arr = np.repeat(TL_vacuum, number)
TL_ground_arr = np.repeat(TL_ground, number)
ground_arr = np.repeat('ground', number)
updown = ['up', 'down']
updown_arr = np.tile(updown, number)

[8.78651685e+09 8.31460674e+09 8.98876404e+09 8.92134831e+09
 8.82022472e+09 8.24719101e+09 8.52808989e+09 8.30337079e+09
 8.44943820e+09 8.16853933e+09 8.89887640e+09 8.49438202e+09
 8.05617978e+09 8.40449438e+09 8.22471910e+09 8.50561798e+09
 8.62921348e+09 8.12359551e+09 8.04494382e+09 8.56179775e+09
 8.85393258e+09 8.96629213e+09 8.29213483e+09 8.43820225e+09
 8.97752809e+09 8.87640449e+09 8.71910112e+09 8.38202247e+09
 8.76404494e+09 8.58426966e+09 8.03370787e+09 8.91011236e+09
 8.14606742e+09 8.88764045e+09 8.01123596e+09 8.83146067e+09
 8.32584270e+09 8.60674157e+09 8.41573034e+09 8.11235955e+09
 8.35955056e+09 8.47191011e+09 8.48314607e+09 8.33707865e+09
 8.86516854e+09 8.66292135e+09 8.93258427e+09 8.28089888e+09
 8.15730337e+09 8.46067416e+09 8.51685393e+09 9.00000000e+09
 8.17977528e+09 8.26966292e+09 8.07865169e+09 8.00000000e+09
 8.10112360e+09 8.53932584e+09 8.75280899e+09 8.59550562e+09
 8.42696629e+09 8.95505618e+09 8.64044944e+09 8.68539326e+09
 8.73033708e+09 8.707865

In [5]:
sample = Sample(2)

sample.Generate_Pads(10000, 5000, #width, depth
                     2, #number of Pads
                     [TL_inner, TL_inner], #d
                     [TL_inner +2*TL_vacuum, TL_inner +2*TL_vacuum], #d1
                     [TL_total, TL_total], #d2
                     0, #layer
                     [coordinates(800, 3900-TL_total/2), coordinates(9200, 1100-TL_total/2)]) #reference coordinates for Pads
sample.Generate_TL(0, 1, TL_inner, TL_inner +2*TL_vacuum, TL_inner + 2*(TL_vacuum+TL_ground), 0)

sample.Generate_Resonators(positions, # ref coordinates
                           f_arr, #freqs
                           updown_arr, #mode
                           w_arr, #w
                           s_arr, #s
                           TL_inner_arr,
                           TL_vacuum_arr,
                           TL_ground_arr,
                           DE_arr) 


coupling length: 375.3680233479264
x coordinate of end of resonator 1356.7680233479264
DE= 11
...
coupling length: 375.3680233479264
x coordinate of end of resonator 1356.7680233479264
DE= 11
...
coupling length: 375.3680233479264
x coordinate of end of resonator 1356.7680233479264
DE= 11
...
coupling length: 375.3680233479264
x coordinate of end of resonator 1356.7680233479264
DE= 11
...
coupling length: 396.67269494334926
x coordinate of end of resonator 1378.0726949433492
DE= 11
...
coupling length: 396.67269494334926
x coordinate of end of resonator 1378.0726949433492
DE= 11
...
coupling length: 396.67269494334926
x coordinate of end of resonator 1378.0726949433492
DE= 11
...
coupling length: 396.67269494334926
x coordinate of end of resonator 1378.0726949433492
DE= 11
...
coupling length: 366.9222428225979
x coordinate of end of resonator 1918.322242822598
DE= 11
...
coupling length: 366.9222428225979
x coordinate of end of resonator 1918.322242822598
DE= 11
...
coupling length: 3

coupling length: 397.7476886965833
x coordinate of end of resonator 7649.147688696584
DE= 11
...
coupling length: 397.7476886965833
x coordinate of end of resonator 7649.147688696584
DE= 11
...
coupling length: 397.7476886965833
x coordinate of end of resonator 7649.147688696584
DE= 11
...
coupling length: 390.8625755766689
x coordinate of end of resonator 7642.262575576669
DE= 11
...
coupling length: 390.8625755766689
x coordinate of end of resonator 7642.262575576669
DE= 11
...
coupling length: 390.8625755766689
x coordinate of end of resonator 7642.262575576669
DE= 11
...
coupling length: 390.8625755766689
x coordinate of end of resonator 7642.262575576669
DE= 11
...
coupling length: 367.3814696596726
x coordinate of end of resonator 8188.781469659672
DE= 11
...
coupling length: 367.3814696596726
x coordinate of end of resonator 8188.781469659672
DE= 11
...
coupling length: 367.3814696596726
x coordinate of end of resonator 8188.781469659672
DE= 11
...
coupling length: 367.381469659

coupling length: 372.0377620507965
x coordinate of end of resonator 5223.437762050797
DE= 11
...
coupling length: 372.0377620507965
x coordinate of end of resonator 5223.437762050797
DE= 11
...
coupling length: 380.723468557819
x coordinate of end of resonator 5232.123468557819
DE= 11
...
coupling length: 380.723468557819
x coordinate of end of resonator 5232.123468557819
DE= 11
...
coupling length: 380.723468557819
x coordinate of end of resonator 5232.123468557819
DE= 11
...
coupling length: 380.723468557819
x coordinate of end of resonator 5232.123468557819
DE= 11
...
coupling length: 369.22992988437534
x coordinate of end of resonator 5800.629929884375
DE= 11
...
coupling length: 369.22992988437534
x coordinate of end of resonator 5800.629929884375
DE= 11
...
coupling length: 369.22992988437534
x coordinate of end of resonator 5800.629929884375
DE= 11
...
coupling length: 369.22992988437534
x coordinate of end of resonator 5800.629929884375
DE= 11
...
coupling length: 398.287373484

coupling length: 400.460838005564
x coordinate of end of resonator 2861.8608380055643
DE= 11
...
coupling length: 400.460838005564
x coordinate of end of resonator 2861.8608380055643
DE= 11
...
coupling length: 392.95554786891347
x coordinate of end of resonator 3424.3555478689136
DE= 11
...
coupling length: 392.95554786891347
x coordinate of end of resonator 3424.3555478689136
DE= 11
...
coupling length: 392.95554786891347
x coordinate of end of resonator 3424.3555478689136
DE= 11
...
coupling length: 392.95554786891347
x coordinate of end of resonator 3424.3555478689136
DE= 11
...
coupling length: 407.6913809139979
x coordinate of end of resonator 3439.091380913998
DE= 11
...
coupling length: 407.6913809139979
x coordinate of end of resonator 3439.091380913998
DE= 11
...
coupling length: 407.6913809139979
x coordinate of end of resonator 3439.091380913998
DE= 11
...
coupling length: 407.6913809139979
x coordinate of end of resonator 3439.091380913998
DE= 11
...
coupling length: 382.7

In [6]:
#gdspy.LayoutViewer(depth = 2)

In [7]:
sample.Generate_Grid(10000,5000,0,0)

In [8]:
gdspy.write_gds('resonators3_1_new.gds', cells=None, name='library', unit=1e-06, precision=1e-09, timestamp=None, binary_cells=None)